In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import time
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor

from ngboost import NGBRegressor

import properscoring as prscore

import pickle
from pathlib import Path
import os

## Read and preprocess the dataset

In [ ]:
df = pd.read_csv('power_weather_data.csv')

# csv file MUST contain 'date' and 'Power' fields
# optional: weather data

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y %H:%M')

In [ ]:
df['hour'] = df['date'].apply(lambda x: x.hour )
df['month'] = df['date'].apply(lambda x: x.month)

In [ ]:
# df['hour_sin'] = np.sin(df['hour'] * 2 * np.pi/24)
# df['hour_cos'] = np.cos(df['hour'] * 2 * np.pi/24)
df['month_sin'] = np.sin(df['month'] * 2 * np.pi/12)
df['month_cos'] = np.cos(df['month'] * 2 * np.pi/12)

In [ ]:
df = df[(df['hour']>=6) & (df['hour']<=21)]

In [ ]:
# df = df.drop(['hour', 'month'], axis=1)
df = df.drop(['month'], axis=1)

In [ ]:
P = df['Power']

PowerData = pd.concat([P.shift(3), P.shift(2), P.shift(1)], axis=1)
PowerData.columns = ['t-45', 't-30', 't-15']

df = pd.concat([df, PowerData.reindex(df.index)], axis=1)
    
df = df.fillna(0)

## Hyperparameters

In [ ]:
weeks = [['2018-03-01', '2019-03-15']]

val_days = 14

# n_points_day = 4 * 24
n_points_day = 4 * 16

## Set the dataframes

In [ ]:
dfs = []

for w in weeks:
    
    w_start = datetime.strptime(w[0]+" 00:00", '%Y-%m-%d %H:%M')
    w_end = datetime.strptime(w[1]+" 23:59", '%Y-%m-%d %H:%M')
    
    dfs.append(df[(df['date'] > w_start) & (df['date'] < w_end)])
    
n_sets = len(dfs)

## Train Test Split

In [ ]:
X_train_ = []
X_test_ = []
y_train_ = []
y_test_ = []

x_scaler = []
y_scaler = []

t_train = []
t_test = []

for i in range(n_sets):

    train = dfs[i][:int(-n_points_day*val_days)]
    test = dfs[i][int(-n_points_day*val_days):]
    
    X_tr = train.drop(['Power','date'], axis=1).values
    X_t = test.drop(['Power','date'], axis=1).values
    
    y_tr = train['Power'].values
    y_t = test['Power'].values
    
    x_sc = MinMaxScaler()
    y_sc = MinMaxScaler()
#     x_sc = StandardScaler()
#     y_sc = StandardScaler()
    x_sc.fit(X_tr)
    y_sc.fit(y_tr.reshape(-1, 1))  #reshape only because fit needs a 2d array
    x_scaler.append(x_sc)
    y_scaler.append(y_sc)
    
    X_train_.append(x_sc.transform(X_tr))
    X_test_.append(x_sc.transform(X_t))
    y_train_.append(y_sc.transform(y_tr.reshape(-1, 1)))
    y_test_.append(y_sc.transform(y_t.reshape(-1, 1)))
    
    t_train.append(dfs[i].iloc[:int(-n_points_day*val_days)]['date'].values)
    t_test.append(dfs[i].iloc[int(-n_points_day*val_days):]['date'].values)

In [ ]:
X_train = X_train_
X_test = X_test_
y_train = y_train_
y_test = y_test_

## NGBoost

In [ ]:
tree_learner = DecisionTreeRegressor(
    criterion="friedman_mse",
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=3,
    splitter="best",
    random_state=None,
)

In [ ]:
ngbs = []

start = time.time()

for i in range(n_sets):
    
    X_train_i = X_train[i]
    y_train_i = y_train[i]

    ngb = NGBRegressor(Base=tree_learner, n_estimators=1000).fit(X_train_i, y_train_i.ravel())
    
    ngbs.append(ngb)

end = time.time()
print((end - start)/n_sets)

## Evaluation

In [ ]:
y = []
y_hat = []
upper_hat = []
lower_hat = []

for i in range(n_sets):
    
    ngb = ngbs[i]
    X_test_i = X_test[i]
    y_test_i = y_test[i]
    
    # For multi-step ahead prediction
    y_first = ngb.predict(X_test_i[:3])
    
    y_3 = y_first[3-3]
    y_2 = y_first[3-2]
    y_1 = y_first[3-1]
    for j in range(3, X_test[i].shape[0]):
        X_test_i[j][-3] = y_3
        X_test_i[j][-2] = y_2
        X_test_i[j][-1] = y_1
        y_pred_j = ngb.pred_dist(X_test_i[j].reshape(1, -1)).loc
        y_3 = y_2
        y_2 = y_1
        y_1 = y_pred_j
    # end of multi-step ahead
    
    y_pred = ngb.predict(X_test_i)
    y_dists = ngb.pred_dist(X_test_i)
    
    mean = y_dists.loc
    std = y_dists.scale
    
    mean = y_scaler[i].inverse_transform(mean.reshape(1, -1))
    std = y_scaler[i].inverse_transform(std.reshape(1, -1))
    mean = mean.flatten()
    std = std.flatten()
    
    real_y_test = y_scaler[i].inverse_transform(y_test_i)
    real_y_test = real_y_test.flatten()
    
    lower = []
    upper = []
    for s in range(1,4):
        lower = lower + [mean - s * std]
        upper = upper + [mean + s * std]
    
    y_hat.append(mean)
    y.append(real_y_test)
    lower_hat.append(lower)
    upper_hat.append(upper)
    
    # Deterministic metrics
    MAE = mean_absolute_error(real_y_test, mean)
    RMSE = mean_squared_error(real_y_test, mean, squared=False)
    MBE = np.mean(mean - real_y_test)
    print(f'MAE: {MAE:.3f}')
    print(f'RMSE: {RMSE:.3f}')
    print(f'MBE: {MBE:.3f}')
    
    # Probabilistic metrics
    PICP = PICP_func(real_y_test, lower[1], upper[1])
    PINAW = PINAW_func(real_y_test, lower[1], upper[1])
    C = prscore.crps_gaussian(real_y_test, mu=mean, sig=std)
    CRPS = C.mean()
    print(f'PICP: {PICP:.3f}')
    print(f'PINAW: {PINAW:.3f}')
    print(f'CRPS: {CRPS:.3f}')
    print('\n') 

## SHAP

In [ ]:
import shap
shap.initjs()

In [ ]:
i = 0
ngb = ngbs[i]

features = list(dfs[i].columns)[2:]

explainer = shap.TreeExplainer(ngb, model_output=0)  # menan (point forecast): model_output=0, std (uncertainty):  model_output=1   
shap_values = explainer.shap_values(X_train[i])

## SHAP Summary Plots

In [ ]:
%matplotlib notebook
shap.summary_plot(shap_values, X_train[i], feature_names=features, show=True, plot_size=(15,8))

In [ ]:
%matplotlib notebook
shap.summary_plot(shap_values, X_train[i], feature_names=features, show=True, plot_size=(15,8), plot_type='bar')

## SHAP Interaction Plots

In [ ]:
# Feature indeces:
# 0: Temperature
# 1: Humidity
# 2: precipitation
# 3: wind speed
# 4: radiation
# 5: hour
# 6: month_sin
# 7: month_cos
# 8: t-45
# 9: t-30
# 10: t-15

In [ ]:
shap_interaction_values = explainer.shap_interaction_values(X_train[i])

In [ ]:
%matplotlib inline
shap.dependence_plot((10,4), shap_interaction_values, X_tr, feature_names=features, ax=ax)

## Force plots

In [ ]:
%matplotlib notebook
shap.force_plot(explainer.expected_value, shap_values[851,:], features=features,link='logit', matplotlib=True, figsize=(10, 3),contribution_threshold=0.025 )

In [ ]:
dfs[i].iloc[851]